# AOC 2022

In [220]:
\cd notebooks

'io



# Advent of Code 2022 - 05

URL: https://adventofcode.com/2022/day/5

We'll study ngn's [solution](https://codeberg.org/ngn/k/src/branch/master/aoc/22/05.k):

[discord discussion](https://discord.com/channels/821509511977762827/821511172305846322/1049371710710894692)

And this one is going to be tough.

In [476]:
x:0:"i/05"
e:x?"" /index of empty line
c:(^:)_'(+e#x)1+4*!9 /crates stacks as top-to-bottom strings (including digits)
(m;f;t):0 -1 -1+`I$(+" "\'1_e_x)1 3 5 /(move;from;to)
h:{(x_y;(x#y),z)} /move x items from the beginning of list y to the beginning of list z
g:`0:*'c{[c;m;f;t]@[c;f,t;:;h[m;c f;c t]]}/
g[1;{m}#f;{m}#t] /part1
g[m;f;t]         /part2

TLNGFGMFN
FGLQJCMBD


First line is pretty clear

In [3]:
x?"" /index of the empty line

9


To take the different stacks, the phrase is the folowing:
- From the lines that contain the stacks. (`e#x`)
- Transpose it (`+`)
- We have 9 stacks, so take 9 (`!9`) lines
- We want one every 4 lines (`4*`)
- Starting at 1. 1,5,9,... (`1+`)
- For each of those, (`'`)
- remove whitespace (`(^:)_`)

In [4]:
(^:)_'(+e#x)1+4*!9

("GBDCPR1"
 "GVH2"
 "MPJDQSN3"
 "MNCDGLSP4"
 "SLFPCNBJ5"
 "STGVZDBQ6"
 "QTFHMZB7"
 "FBDMC8"
 "GQCF9")


In [5]:
*1_e_x

"move 1 from 7 to 4"


Now, for each command line, we want to get the relevant information of `move`, `from`, and `to`:

In [7]:
(m;f;t):0 -1 -1+`I$(+" "\'1_e_x)1 3 5 /(move;from;to)
(*m; *f; *t)

1 6 3


Explanation:

- Take the lines that have move commands (`1_e_x`)
- Split each line by spaces (`" "\'`)
- Transpose (`flip`) the full list, to be able to select the desired fields `1, 3, and 5` by leading axis ([citation needed] https://aplwiki.com/wiki/Leading_axis_theory)
- Select rows 1,3,5
- convert everything to ints ``` `I$```
- subtract 1 from `from` and `to`, so the numbers map to our 0-index nature

In [23]:
/ why the `flip`
(" "\'5#1_e_x)
(+" "\'5#1_e_x)

(("move";,"1";"from";,"7";"to";,"4")
 ("move";,"3";"from";,"4";"to";,"7")
 ("move";,"4";"from";,"3";"to";,"4")
 ("move";,"5";"from";,"6";"to";,"9")
 ("move";,"1";"from";,"8";"to";,"1"))
(("move";"move";"move";"move";"move")
 (,"1";,"3";,"4";,"5";,"1")
 ("from";"from";"from";"from";"from")
 (,"7";,"4";,"3";,"6";,"8")
 ("to";"to";"to";"to";"to")
 (,"4";,"7";,"4";,"9";,"1"))


`h` is a simple function to shift `x` number of elements elements from `y` to `z`, preserving the order. 

In [478]:
h:{(x_y;(x#y),z)}

In [145]:
h[2;0 1 2 3; 4 5 6]

(2 3
 0 1 4 5 6)


Now this starts to be fun again:

In [156]:
g:`0:*'c{[c;m;f;t]@[c;f,t;:;h[m;c f;c t]]}/

This line defines a function `g`. If we split it into meaningful parts, we get:
- Define function `g:`
- print the first of each ``` `0:*'```
- over  (`c f /`)
- `f` being a tetradic function
- And what `f` does is to `Amend` `c`, in the indices `f,t`, setting as values (`:`), the result of `h` `@[c;f,t;:;h[m;c f;c t]]`

A function definition that ends with `/` tells us that the full body will be applied to the parameters it'll get when called. It's kind of a literal, that could be inlined.

In [36]:
ff:`0:*'c{[c;m;f;t]@[c;f,t;:;h[m;c f;c t]]}/
ff[1; (1 2 3); (1 2 3)]

GGMMSSQFG


## Detour #1, `over`

We'll digress a bit, and write a few tests on `over` with a simple triadic function, to see how it behaves depending on the shapes of the arguments

Reading the holy [kreflite.pdf](http://www.nsl.com/k/k2/k295/kreflite.pdf) doc on `over`:

[...]each argument other than the first can be an atom or list,
but if two or more of those are lists then all those lists must conform. The first
argument x is any valid first argument of f.

The general case of Over for functions with valence at least two is a direct exten-
sion of the dyadic case. The first argument x serves the same role as the left argu-
ment of the dyadic case, and all other arguments have the same role as the right
argument. That is, if all of y, z,... are atoms then f/[x;y;z;...] is identical to
f[x;y;z;...] , and otherwise f/[x;y;z;...] is evaluated as follows:
```
x: f[x;y0;z0;...]
x: f[x;y1;z1;...]
.
.
.
x: f[x;yn;zn;...]
```
where y0 is y[0] if y is a list or y itself if y is an atom, and similarly for y1...yn,
z0...zn. The name yn stands for the last item of y if y is a list or y itself if y is an
atom, and similarly for zn, ... . The result is the last value of x.

And, as usual in K, this description holds a lot of non-trivial behaviors that you can exploit when needed. Let's see a few examples with a triadic sum `{x+y+z}`:

In [466]:
`simple
{x+y+z}/[2;1;0]       / same as {x+y+z}[2;1;0] (2+1+0)
{x+y+z}/[2 1;1;0]     / same as {x+y+z}[2 1;1;0]  (2 1 + 1 + 0)
{x+y+z}/[2 1 0;1;0]   / same as {x+y+z}[2 1 0;1;0]  (2 1 0 + 1 + 0)
/so far, no iterations have happened on over. Everything happened in one call of the function, and as `+` is atomic, everything just works.

`iterate
{x+y+z}/[0;2 1 0;3]   / 0+2+3  + 1+3  + 0+3. scalar, because first arg is scalar

`conform
{x+y+z}/[2 1 0;1 0;3] / `1 0` and `3` conform.
{x+y+z}/[2 1 0;1 0;3 0] / `1 0` and `3 0` conform.
{x+y+z}/[2 1 0;1 0;3 0 0] / `1 0` and `3 0 0` do not conform. 'lenght error

`LeftfirstArg 
p:1
3{y}/[0;1;3]

`recurrence
{x+y+z}\[10;2;1;3]    / recurrence
{x+y}\[10;1;1] / fib 
+\[10;1;1] / also fib

`simple
3
3 2
3 2 1
`iterate
12
`conform
9 8 7
6 5 4
'length
 {x+y+z}/[2 1 0;1 0;3 0 0] / `1 0` and `3 0 0` do not conform. 'lenght error
                         ^

`LeftfirstArg
'rank
 3{y}/[0;1;3]
  ^

`recurrence
2 1 3 6 10 19 35 64 118 217 399 734 1350
1 1 2 3 5 8 13 21 34 55 89 144
1 1 2 3 5 8 13 21 34 55 89 144


In [234]:
p:1
 3{y+x+z}\[2 3;4 3]

'rank
  3{y+x+z}\[2 3;4 3]
   ^



The last section (recurrence) is an undocumented extension to ngn/k:

ngn: [...]I made a small undocumented extension to k6 in my implementation: dyad\[nIterations;seed1;seed2] computes the first n terms of a recurrence relation, e.g. +\[10;1;1] is the fibonacci sequence. [discord link](https://discord.com/channels/821509511977762827/821511172305846322/1065755527734370346).

If you're wondering about the behaviors on the valence of `f` vs number of args given to `f/`, [that's your link](https://ngn.codeberg.page/txt/slash.html)

## Detour #2: Tables indexing tables

In [481]:
h[m;c f;c t]


'limit
 {(x_y;(x#y),z)}
         ^
 h[m;c f;c t]
            ^



In [474]:
pp:c{[c;m;f;t] @[c;f,t;:;h[m;c f;c t]]}/
pp[1; 1 2; 1 2]

/c{[c;m;f;t] @[c;f,t;:;h[m;c f;c t]]}/[1; 1 2; 1 2]   / fails

c:42
pp:c{[c;m;f] c}/
pp[1;1]


'rank
 {[c;m;f;t] @[c;f,t;:;h[m;c f;c t]]}
                                ^
 pp[1; 1 2; 1 2]
               ^

42


In [127]:
10#m
10#f
10#m f
10#{m}#f

1 3 4 5 1 2 3 1 9 1
6 3 2 5 7 2 3 2 6 1
3 5 4 2 1 4 5 4 3 3
6 3 3 3 2 2 2 2 5 5


Function `g` looks dauntingly big, right? Well, it's not that bad, it's just a user defined tetradic function, meaning we had to define the parameter list instead of relying on `x,y,z`.

This is a case of `over`, not `over monad` nor `over dyad`. 

In [318]:
 /{a:1; {[a;x] a+x}[a]@2} 1
 / gg:`0:*'c{[c;m;f;t] c}/[1 2; 3 4; 5 6]
#{m}#t
#{m}#f
 /ff:[1;{m}#f;{m}#t] 

2383
2383


In [484]:
g:`0:*'c{[c;m;f;t]@[c;f,t;:;h[m;c f;c t]]}/
ff:($!10){[c;m;f;t]@[c;f,t;:;h[m;c f;c t]]}/
ff[1;2;3]
 /g[2;1;2]

 /*{[c;m;f;t]@[c;f,t;:;h[m;c f;c t]]}/[1;{m}#f;{m}#t]
 g[1;{m}#f;{m}#t] /part1

(,"0"
 ,"1"
 ""
 "23"
 ,"4"
 ,"5"
 ,"6"
 ,"7"
 ,"8"
 ,"9")
TLNGFGMFN
